<a href="https://colab.research.google.com/github/LuiLazarim/Imerss-o_IA_projeto_ensalamento/blob/main/Ensalamento_professores__automatizado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importando bibliotecas

In [46]:
!pip install -q -U google-generativeai

In [47]:
import pandas as pd
import numpy as np
import google.generativeai as genai
from google.colab import userdata

api_key = userdata.get('google-generativeai-api-key')

GOOGLE_API_KEY = api_key
genai.configure(api_key=GOOGLE_API_KEY)

In [48]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [49]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


#Obtendo o banco de dados

Os dados são apenas hipotéticos para um teste em pequena escala, podem ser obtidos por muitos modos, e sua estrutura concatenada pode ser automatizada de acordo com informaçoes tabuladas das disciplinas, cursos, periodos, professores e outros que estejam relacionados, assim dando mais informação e precisão para o modelo.

In [50]:
disciplinas = [
    {'disciplina':'comunicação e expressão','conteudo': 'disciplina -> comunicação e expressão, hora aula -> 1, pode ocorrer na sala 1/2/3, cursos -> historia(campus -> paulista(matutino - 10 alunos)/mooca(matutino - 13 alunos/noturno - 11 alunos))/letras(campus paulista(noturno - 13 alunos)/mooca(matutino - 13 alunos)/augusta(matutino - 11 alunos/noturno - 17 alunos))/matematica(campus paulista(matutino - 14 alunos/noturno - 5 alunos)/mooca(matutino - 13 alunos))/augusta(matutino - 19 alunos/noturno - 15 alunos))/engenharia(campus paulista(matutino - 15 alunos/noturno - 15 alunos)/mooca(matutino - 13 alunos/noturno - 18 alunos)/augusta(matutino - 23 alunos/noturno - 29 alunos))biologia(campus paulista(noturno - 16 alunos)/mooca(matutino - 19 alunos/noturno - 22 alunos)/augusta(matutino - 15 alunos/noturno - 12 alunos)), professores com aderencia -> ana(disponibilidade - segunda(Matutino),terça(Matutino/Noturno),quinta(Noturno),sexta(Matutino/Noturno))/monica(disponibilidade - segunda(Noturno),terça(Matutino/Noturno),quarta(Matutino/Noturno),quinta(Matutino/Noturno))/gustavo(disponibilidade - terça(Matutino/Noturno),quarta(Noturno),quinta(Matutino/Noturno),sexta(Matutino/Noturno))/luis(disponibilidade - segunda(Matutino/Noturno),quarta(Matutino/Noturno),quinta(Matutino),sexta(Matutino/Noturno))/juliana(disponibilidade - segunda(Matutino/Noturno),terça(Matutino/Noturno),quarta(Matutino/Noturno),quinta(Matutino/Noturno),sexta(Matutino))/renan(disponibilidade - terça(Matutino/Noturno),quinta(Matutino/Noturno))'},
    {'disciplina':'calculo','conteudo': 'disciplina -> calculo, hora aula -> 3, pode ocorrer na sala 3, cursos -> matematica(campus -> paulista(matutino - 14 alunos/noturno - 5 alunos)/mooca(matutino - 13 alunos))/augusta(matutino - 19 alunos/noturno - 15 alunos))/engenharia(campus paulista(matutino - 15 alunos/noturno - 15 alunos)/mooca(matutino - 13 alunos/noturno - 18 alunos)/augusta(matutino - 23 alunos/noturno - 29 alunos))biologia(campus paulista(noturno - 16 alunos)/mooca(matutino - 19 alunos/noturno - 22 alunos)/augusta(matutino - 15 alunos/noturno - 12 alunos)), professores com aderencia -> gustavo(disponibilidade - terça(Matutino/Noturno),quarta(Noturno),quinta(Matutino/Noturno),sexta(Matutino/Noturno))/luis(disponibilidade - segunda(Matutino/Noturno),quarta(Matutino/Noturno),quinta(Matutino),sexta(Matutino/Noturno))/juliana(disponibilidade - segunda(Matutino/Noturno),terça(Matutino/Noturno),quarta(Matutino/Noturno),quinta(Matutino/Noturno),sexta(Matutino))/renan(disponibilidade - terça(Matutino/Noturno),quinta(Matutino/Noturno))'},
    {'disciplina':'primeiros socorros','conteudo': 'disciplina -> primeiros socorros, hora aula -> 1, pode ocorrer na sala 1, cursos -> historia(campus -> paulista(matutino - 10 alunos)/mooca(matutino - 13 alunos/noturno - 11 alunos))/letras(campus paulista(noturno - 13 alunos)/mooca(matutino - 13 alunos)/augusta(matutino - 11 alunos/noturno - 17 alunos))/engenharia(campus paulista(matutino - 15 alunos/noturno - 15 alunos)/mooca(matutino - 13 alunos/noturno - 18 alunos)/augusta(matutino - 23 alunos/noturno - 29 alunos))biologia(campus paulista(noturno - 16 alunos)/mooca(matutino - 19 alunos/noturno - 22 alunos)/augusta(matutino - 15 alunos/noturno - 12 alunos)), professores com aderencia -> luis(disponibilidade - segunda(Matutino/Noturno),quarta(Matutino/Noturno),quinta(Matutino),sexta(Matutino/Noturno))/juliana(disponibilidade - segunda(Matutino/Noturno),terça(Matutino/Noturno),quarta(Matutino/Noturno),quinta(Matutino/Noturno),sexta(Matutino))/renan(disponibilidade - terça(Matutino/Noturno),quinta(Matutino/Noturno))'},
    {'disciplina':'estatistica','conteudo': 'disciplina -> estatistica, hora aula -> 3, pode ocorrer na sala 3, cursos -> historia(campus -> paulista(matutino - 10 alunos)/mooca(matutino - 13 alunos/noturno - 11 alunos))/matematica(campus paulista(matutino - 14 alunos/noturno - 5 alunos)/mooca(matutino - 13 alunos))/augusta(matutino - 19 alunos/noturno - 15 alunos))/engenharia(campus paulista(matutino - 15 alunos/noturno - 15 alunos)/mooca(matutino - 13 alunos/noturno - 18 alunos)/augusta(matutino - 23 alunos/noturno - 29 alunos))biologia(campus paulista(noturno - 16 alunos)/mooca(matutino - 19 alunos/noturno - 22 alunos)/augusta(matutino - 15 alunos/noturno - 12 alunos)), professores com aderencia -> ana(disponibilidade - segunda(Matutino),terça(Matutino/Noturno),quinta(Noturno),sexta(Matutino/Noturno))/monica(disponibilidade - segunda(Noturno),terça(Matutino/Noturno),quarta(Matutino/Noturno),quinta(Matutino/Noturno))/gustavo(disponibilidade - terça(Matutino/Noturno),quarta(Noturno),quinta(Matutino/Noturno),sexta(Matutino/Noturno))'},
    {'disciplina':'sociologia','conteudo': 'disciplina -> sociologia, hora aula -> 2, pode ocorrer na sala 1/2/3, cursos -> historia(campus -> paulista(matutino - 10 alunos)/mooca(matutino - 13 alunos/noturno - 11 alunos))/letras(campus paulista(noturno - 13 alunos)/mooca(matutino - 13 alunos)/augusta(matutino - 11 alunos/noturno - 17 alunos))/matematica(campus paulista(matutino - 14 alunos/noturno - 5 alunos)/mooca(matutino - 13 alunos))/augusta(matutino - 19 alunos/noturno - 15 alunos)), professores com aderencia -> monica(disponibilidade - segunda(Noturno),terça(Matutino/Noturno),quarta(Matutino/Noturno),quinta(Matutino/Noturno))/gustavo(disponibilidade - terça(Matutino/Noturno),quarta(Noturno),quinta(Matutino/Noturno),sexta(Matutino/Noturno))/luis(disponibilidade - segunda(Matutino/Noturno),quarta(Matutino/Noturno),quinta(Matutino),sexta(Matutino/Noturno))'},
]

disciplinas_df = pd.DataFrame(disciplinas)
disciplinas_df

,disciplina,conteudo
0,comunicação e expressão,"disciplina -> comunicação e expressão, hora au..."
1,calculo,"disciplina -> calculo, hora aula -> 3, pode oc..."
2,primeiros socorros,"disciplina -> primeiros socorros, hora aula ->..."
3,estatistica,"disciplina -> estatistica, hora aula -> 3, pod..."
4,sociologia,"disciplina -> sociologia, hora aula -> 2, pode..."


#Definindo embaddings

In [51]:
model = 'models/embedding-001'

In [52]:
def embed_fn(disciplina, conteudo):
  return genai.embed_content(model=model,
                                 content=conteudo,
                                 title=disciplina,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [53]:
disciplinas_df["embeddings"] = disciplinas_df.apply(lambda row: embed_fn(row["disciplina"],
                                                                           row["conteudo"]),
                                                      axis=1)
disciplinas_df

,disciplina,conteudo,embeddings
0,comunicação e expressão,"disciplina -> comunicação e expressão, hora au...","[0.052909542, -0.028234096, 0.0047384296, 0.01..."
1,calculo,"disciplina -> calculo, hora aula -> 3, pode oc...","[0.031012757, -0.021681812, -0.017606765, 0.01..."
2,primeiros socorros,"disciplina -> primeiros socorros, hora aula ->...","[0.04533144, -0.010821828, -0.0026816912, 0.02..."
3,estatistica,"disciplina -> estatistica, hora aula -> 3, pod...","[0.055508856, -0.043072082, -0.026183568, 0.03..."
4,sociologia,"disciplina -> sociologia, hora aula -> 2, pode...","[0.060968574, -0.014856817, -0.022252202, 0.01..."


In [54]:
def ensalamento(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(disciplinas_df["embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return disciplinas_df.iloc[indice]["conteudo"]

#Configurando o Modelo AI e determinando variáveis

Optei por um modelo com temperatura mediana por enquanto, pois é necessário uma criatividade para fazer combinações no ensalamento para obter o melhor resultado, no entanto não pode fugir dos parâmetros definidos.

In [70]:
generation_config = {
    "candidate_count": 1,
    "temperature": 0.5,
    "top_k": 40,
    "top_p": 0.95
}

In [56]:
safety_settings = {
    "HARASSMENT": "BLOCK_SOME",
    "HATE" : "BLOCK_SOME",
    "SEXUAL": "BLOCK_SOME",
    "DANGEROUS": "BLOCK_SOME"
}

In [57]:
disciplina = [
    'comunicação e expressão',
    'calculo',
    'primeiros socorros',
    'estatistica',
    'sociologia'
]

In [58]:
modelAI = genai.GenerativeModel("gemini-1.0-pro",
                                  generation_config = generation_config)

In [59]:
def criar_tabela(parametros, colunas, prompt, observacoes):
  prompt = f"{prompt}. {parametros}. {colunas}. {observacoes}"
  response = modelAI.generate_content(prompt)
  return print(response.text)

In [60]:
dados = []

for i in range(0, len(disciplina)):
  busca = f'disciplina {disciplina[i]}'
  consulta = busca
  resposta = ensalamento(consulta, disciplinas_df, model)
  dados.append(resposta)

  i = i + 1

print(dados)

['disciplina -> comunicação e expressão, hora aula -> 1, pode ocorrer na sala 1/2/3, cursos -> historia(campus -> paulista(matutino - 10 alunos)/mooca(matutino - 13 alunos/noturno - 11 alunos))/letras(campus paulista(noturno - 13 alunos)/mooca(matutino - 13 alunos)/augusta(matutino - 11 alunos/noturno - 17 alunos))/matematica(campus paulista(matutino - 14 alunos/noturno - 5 alunos)/mooca(matutino - 13 alunos))/augusta(matutino - 19 alunos/noturno - 15 alunos))/engenharia(campus paulista(matutino - 15 alunos/noturno - 15 alunos)/mooca(matutino - 13 alunos/noturno - 18 alunos)/augusta(matutino - 23 alunos/noturno - 29 alunos))biologia(campus paulista(noturno - 16 alunos)/mooca(matutino - 19 alunos/noturno - 22 alunos)/augusta(matutino - 15 alunos/noturno - 12 alunos)), professores com aderencia -> ana(disponibilidade - segunda(Matutino),terça(Matutino/Noturno),quinta(Noturno),sexta(Matutino/Noturno))/monica(disponibilidade - segunda(Noturno),terça(Matutino/Noturno),quarta(Matutino/Noturn

#Prompt para geração do ensalamento

O prompt deve gerar uma combinatória entre professores com disponibilidade no campus, periodo e dia da semana e aderêncioa a disciplina, informando as possibilidades possiveis de acordo com os parametros informados. Isso possibilita definição rápida do ensalamento de acordo com os dados obtidos, evitando choque de turmas e otimizando as turmas de acordo com o número de alunos.

In [71]:
prompt = f"Crie uma tabela com base nos seguintes dados:{dados}"
parametros = 'Professores não podem dar aula no mesmo dia da semana naquele campus e naquele periodo. Caso disciplina tenha menos de 40 alunos, tente unir cursos para somar até 40 alunos se possivel, e informe quais cursos foram unidos'
colunas = 'A tabela deve ter as colunas dia da semana, disciplina, campus, periodo, curso, professor, numero de alunos total'
observacoes = 'Caso haja conflito entre professor e curso informe na coluna professor: não há professor disponivel'

criar_tabela(parametros=parametros, colunas=colunas, prompt=prompt, observacoes=observacoes)

| Dia da Semana | Disciplina | Campus | Periodo | Cursos | Professor | Numero de Alunos |
|---|---|---|---|---|---|---|
| Segunda | Comunicação e Expressão | Paulista | Matutino | Historia (10), Letras (13) | Ana | 23 |
| Segunda | Primeiros Socorros | Mooca | Matutino | Engenharia (15) | Não há professor disponível | 15 |
| Segunda | Sociologia | Augusta | Matutino | Matematica (14), Engenharia (23) | Monica | 37 |
| Terça | Comunicação e Expressão | Paulista | Matutino | Matematica (14) | Não há professor disponível | 14 |
| Terça | Calculo | Paulista | Matutino | Matematica (14), Engenharia (15) | Juliana | 29 |
| Terça | Primeiros Socorros | Augusta | Matutino | Biologia (15) | Não há professor disponível | 15 |
| Terça | Estatistica | Paulista | Noturno | Historia (10), Matematica (5) | Ana | 15 |
| Terça | Sociologia | Mooca | Noturno | Letras (13) | Não há professor disponível | 13 |
| Quarta | Comunicação e Expressão | Augusta | Matutino | Letras (11) | Não há professor disponí